In [1]:
import numpy as np

# Problem 1

In [19]:
def calc_freq(start_freq,input_file):
    freq = start_freq
    with open(input_file,'r') as f:
        for line in f:
            freq += int(line)
    return freq

def calc_freq_repeat(start_freq,input_file):
    freq = start_freq
    repeat = set()
    first_repeat = None
    flines = []
    
    with open(input_file,'r') as f:
        flines = [int(line) for line in f]
        
    while not first_repeat:
        for i in flines:
            freq += i
            if freq in repeat:
                first_repeat = freq
                break
            repeat.add(freq)
    print(first_repeat)
    return freq

In [20]:
calc_freq(0,'input1.txt')

547

In [21]:
calc_freq_repeat(0,'input1.txt')

76414


76414

# Problem 2

In [7]:
import numpy as np

In [11]:
def calc_checksum(name_list):
    twos = 0
    threes = 0
    for name in name_list:
        char_ct = {}
        for char in name:
            char_ct[char] = char_ct.get(char,0) + 1
        twos_check = (len([k for k,v in char_ct.items() if v==2]) > 0)*1
        threes_check = (len([k for k,v in char_ct.items() if v==3]) > 0)*1
        twos += twos_check
        threes += threes_check
    return twos*threes

In [ ]:
          if s[i] = t[j]:
            substitutionCost := 0
          else:
            substitutionCost := 1

          d[i, j] := minimum(d[i-1, j] + 1,                   // deletion
                             d[i, j-1] + 1,                   // insertion
                             d[i-1, j-1] + substitutionCost)  // substitution
 
  return d[m, n]

In [43]:
def lev_dist(s,t):
    m = len(s)
    n = len(t)
    
    d = np.zeros((m,n))
    d[:,0] = range(m)
    d[0,:] = range(n)
    
    for j in range(n):
        for i in range(m):
            if s[i] == t[j]:
                cost = 0
            else:
                cost = 1
            d[i,j] = min(d[i-1,j]+1,
                               d[i,j-1] + 1,
                               d[i-1,j-1] + cost)
        
    return d[m-1,n-1]

In [1]:
name_list = [
    'abcdef',
    'bababc',
    'abbcde',
    'abcccd',
    'aabcdd',
    'abcdee',
    'ababab'
]

In [13]:
calc_checksum(name_list)

12

In [17]:
with open('input2.txt','r') as f:
    name_list = f.readlines()
    name_list = [name.strip() for name in name_list]

In [19]:
calc_checksum(name_list)

5750

In [47]:
for s in name_list:
    for t in name_list:
        if lev_dist(s,t) == 1:
            print(s,t)
            break

tzyvunogzariwkpcbdewsmjhxi tzyvunogzariwkpcbdewmmjhxi


KeyboardInterrupt: 

# Problem 3

In [40]:
class Claim:
    def __init__(self,claim_str):
        # #123 @ 3,2: 5x4
        claim_list = claim_str.replace('@ ','').replace(':','').split(' ')
        
        claim_num = claim_list[0][1:]
        self.claim_num = claim_num
        
        xy = claim_list[1].split(',')
        self.x = int(xy[0])
        self.y = int(xy[1])
        wh = claim_list[2].split('x')
        self.width = int(wh[0])
        self.height = int(wh[1])
        
        self.max_x = self.x + self.width - 1
        self.max_y = self.y + self.height - 1
    
    def get_bounds(self):
        min_x = self.x
        min_y = self.y
        max_x = self.max_x
        max_y = self.max_y
        return (min_x,max_x,min_y,max_y)
    
    def contains_pt(self,pt_x,pt_y):
        in_x = pt_x >= self.x and pt_x <= self.max_x
        in_y = pt_y >= self.y and pt_y <= self.max_y
        
        return in_x and in_y
    
    def get_matrix(self,max_dim):
        full_mat = np.zeros((max_dim,max_dim))
        min_x = self.x
        min_y = self.y
        max_x = self.max_x + 1
        max_y = self.max_y + 1
        full_mat[min_x:max_x,min_y:max_y] = 1
        return full_mat
    
    def get_area(self):
        return self.width * self.height
    
def calculate_all_overlaps(claims):
    max_pt = 1000
    
    full_mat = np.zeros((max_pt,max_pt))
    for c in claims:
        full_mat = full_mat + c.get_matrix(max_pt)
    return full_mat

def calculate_non_overlap(full_matrix,claims):
    for c in claims:
        n = sum(full_matrix[c.get_matrix(1000).nonzero()] > 1)
#         print(n)
        if n == 0:
            print(c.claim_num)
            break

In [3]:
claim_str = '#123 @ 3,2: 5x4'
c = Claim(claim_str)

In [42]:
claim_strs = [
    '#1 @ 1,3: 4x4',
    '#2 @ 3,1: 4x4',
    '#3 @ 5,5: 2x2',
]
claims = [Claim(c) for c in claim_strs]
m = calculate_all_overlaps(claims)
calculate_non_overlap(m,claims)

3


In [43]:
with open('input3.txt','r') as f:
    claim_list = f.readlines()
    claim_list = [Claim(c) for c in claim_list]

In [44]:
m = calculate_all_overlaps(claim_list)
print('Answer: %i'%(np.sum((m >= 2)*1)))

Answer: 107820


In [45]:
calculate_non_overlap(m,claim_list)

661


# Problem 4

In [117]:
from datetime import datetime, timedelta
import re
import operator

class Guard():
    def __init__(self,id_num):
        self.id_num = id_num
        self.days_worked = []
        self.sleep_schedule = []
        
    def work(self,datetime):
        self.days_worked.append(datetime)
        
    def sleep(self,sleep_time):
        self.sleep_schedule.append([sleep_time])
    
    def wake(self,wake_time):
        self.sleep_schedule[-1].append(wake_time)
        
    def get_time_slept(self):
        return self.time_count, self.time_slept
        
    def calculate_time_slept(self):
        self.time_slept = 0
        self.time_count = {}
        for s in self.sleep_schedule:
            sleep = s[0]
            wake = s[1]
            self.time_slept += (wake-sleep).seconds//60
            
            m = timedelta(minutes=1)
            cur_time = sleep
            while cur_time < wake:
                time_part = datetime.strftime(cur_time,'%H-%M')
                self.time_count[time_part] = self.time_count.get(time_part,0) + 1
                cur_time += m
                
        return self.time_count, self.time_slept

# def parse_timestamp(t):
#     dt = datetime.fromisoformat(t)
#     return dt
        
def parse_input(puzzle_input):
    guards = {}
    events = {}
    re_start = re.compile(r'\[([0-9-: ]+)\] Guard #([0-9]+) [a-zA-Z ]+')
    re_sleep = re.compile(r'\[([0-9-: ]+)\] falls asleep')
    re_wake = re.compile(r'\[([0-9-: ]+)\] wakes up')
    
    for p in puzzle_input:
        if re_start.match(p):
            res = re_start.match(p)
            ts = datetime.fromisoformat(res.group(1))
            guardnum = res.group(2)
            g = guards.get(guardnum,Guard(guardnum))
            guards[guardnum] = g
            g.work(ts)
            events[ts] = (g,'sw')
        elif re_sleep.match(p):
            res = re_sleep.match(p)
            ts = datetime.fromisoformat(res.group(1))
            events[ts] = (None,'s')
        elif re_wake.match(p):
            res = re_wake.match(p)
            ts = datetime.fromisoformat(res.group(1))
            events[ts] = (None,'w')
            
    events = {k:v for k,v in sorted(events.items(),key=lambda x: x[0])}
            
    for k,v in events.items():
        if v[1] == 'sw':
            cur_guard = v[0]
        elif v[1] == 's':
            cur_guard.sleep(k)
        else:
            cur_guard.wake(k)
        
    max_guard = None
    max_slept = 0
    max_time_count = None
    for k,g in guards.items():
        tc,ts = g.calculate_time_slept()
        if ts > max_slept:
            max_guard = int(k)
            max_slept = ts
            max_time_count = tc
    
    max_time = max(max_time_count.items(), key=operator.itemgetter(1))[0]
    max_minute = int(max_time.split('-')[1])
    answer_part_one = max_guard * max_minute
    
    max_guard = None
    max_ct = 0
    max_minute = None
    for k,g in guards.items():
        tc,ts = g.get_time_slept()
        if ts > 0:
            slept_max = max(tc.items(), key=operator.itemgetter(1))
            slept_ct = slept_max[1]
            slept_minute = int(slept_max[0].split('-')[1])
            if slept_ct > max_ct:
                max_guard = int(k)
                max_ct = slept_ct
                max_minute = slept_minute
    answer_part_two = max_guard * max_minute
            
    return guards,events,answer_part_one,answer_part_two
    

# puzzle_input = [
#     '[1518-11-01 00:00] Guard #10 begins shift',
#     '[1518-11-01 00:05] falls asleep',
#     '[1518-11-01 00:25] wakes up',
#     '[1518-11-01 00:30] falls asleep',
#     '[1518-11-01 00:55] wakes up',
#     '[1518-11-01 23:58] Guard #99 begins shift',
#     '[1518-11-02 00:40] falls asleep',
#     '[1518-11-02 00:50] wakes up',
#     '[1518-11-03 00:05] Guard #10 begins shift',
#     '[1518-11-03 00:24] falls asleep',
#     '[1518-11-03 00:29] wakes up',
#     '[1518-11-04 00:02] Guard #99 begins shift',
#     '[1518-11-04 00:36] falls asleep',
#     '[1518-11-04 00:46] wakes up',
#     '[1518-11-05 00:03] Guard #99 begins shift',
#     '[1518-11-05 00:45] falls asleep',
#     '[1518-11-05 00:55] wakes up'
# ]
with open('input4.txt','r') as f:
    puzzle_input = [l.strip() for l in f.readlines()]
g,e,a1,a2 = parse_input(puzzle_input)

print(f'Part One: {a1}')
print(f'Part Two: {a2}')

Part One: 38813
Part Two: 141071


# Problem 5

In [127]:
class Node():
    def __init__(self,data):
        self.data = data
        self.prev_node = None
        self.next_node = None
    
    def add_prev(self,prev_node):
        self.prev_node = prev_node
    
    def add_next(self,next_node):
        self.next_node = next_node
        if next_node is not None:
            next_node.add_prev(self)
            r = self.react()
        else:
            r = False
        return r
    
    def add(self,node):
        s = self
        while s.next_node:
            s = s.next_node
        s.add_next(node)
    
    def react(self):
        i = self.data
        j = self.next_node.data
        #same element, react and remove both elements from linked list
        if i.lower()==j.lower() and i!=j:
            next_next = self.next_node.next_node
            self.prev_node.add_next(next_next)
            return True
        else:
            return False
            
    def get_list(self):
        s = self
        d = []
        while s:
            d.append(s.data)
            s = s.next_node
        return d

In [133]:
class LinkedList():
    def __init__(self):
        self.start = Node('start')
        self.last_node = self.start
        
    def add(self,data):
        d = Node(data)
        reacted = self.last_node.add_next(d)
        if reacted:
#             print('REACTED')
            #find last node
            s = self.start
            while s.next_node:
                s = s.next_node
            self.last_node = s
        else:
            self.last_node = self.last_node.next_node
            
    def get_list(self):
        s = self.start.next_node
        d = []
        while s:
            d.append(s.data)
            s = s.next_node
        return d

In [137]:
start = Node('start')
polymer = 'dabAcCaCBAcCcaDA'
s = start
for i in polymer:
    n = Node(i)
    s.add(n)

len(start.get_list()[1:])

10

In [167]:
with open('input5.txt','r') as f:
    polymer = f.readlines()[0].strip()
    
#One method, quick

def collapse_polymer(polymer_string, letter_to_remove=None):
    if letter_to_remove:
        new_polymer_string = [i for i in polymer_string if i.lower()!=letter_to_remove]
    else:
        new_polymer_string = polymer_string
    stack = []
    for i in new_polymer_string:
        if len(stack) == 0:
            stack.append(i)
            continue
        j = stack[-1]
        if i.lower() != j.lower():
            stack.append(i)
        elif i != j:
            _ = stack.pop()
        else:
            stack.append(i)
    return len(stack)
        
#Linked list method, bad and slow

def collapse_polymer_ll(polymer_string):
    l = LinkedList()
    for i in polymer_string:
        l.add(i)
    
    return len(l.get_list())

In [169]:
for collapse_polymer(polymer,'a')

9390

In [172]:
min_len = None
for i in range(97,123):
    a = collapse_polymer(polymer,chr(i))
    if not min_len or a < min_len:
        min_len = a
        
print(min_len)

6484
